Project 2
Michael Bell
Rob Chesser
Adam Durar

In [1]:
import pandas as pd
import numpy as np
import os
import glob
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
#pip install sqlalchemy-utils for the above

Get list of CSV's from each data set

In [2]:
path = './quandl_csv'
all_files = glob.glob(os.path.join(path, "*.csv"))

li = []

for filename in all_files:
    fn, ext = os.path.splitext(os.path.basename(filename))
    df = pd.read_csv(filename, index_col=None, header=0)
    df = df[['Date','Adj_Open','Adj_Close']]
    df = df.rename(columns={"Date": "date", "Adj_Open": "adj_open", "Adj_Close": "adj_close"})
    df['symbol'] = fn
    li.append(df)
li

[             date    adj_open   adj_close symbol
 0      2019-11-15  120.980000  120.760000    AXP
 1      2019-11-14  120.000000  120.930000    AXP
 2      2019-11-13  120.270000  120.260000    AXP
 3      2019-11-12  120.870000  120.890000    AXP
 4      2019-11-11  120.760000  120.900000    AXP
 5      2019-11-08  121.010000  121.470000    AXP
 6      2019-11-07  120.270000  121.280000    AXP
 7      2019-11-06  118.890000  119.690000    AXP
 8      2019-11-05  119.840000  118.770000    AXP
 9      2019-11-04  119.620000  119.620000    AXP
 10     2019-11-01  118.410000  119.140000    AXP
 11     2019-10-31  117.790000  117.280000    AXP
 12     2019-10-30  117.440000  118.070000    AXP
 13     2019-10-29  118.090000  117.420000    AXP
 14     2019-10-28  118.640000  118.580000    AXP
 15     2019-10-25  116.360000  118.260000    AXP
 16     2019-10-24  116.440000  116.410000    AXP
 17     2019-10-23  118.420000  116.490000    AXP
 18     2019-10-22  118.670000  118.610000    AXP


Flatten DF Lists into DF

In [3]:
djdata = pd.concat(li, axis=0, ignore_index=True)
djdata['date'] = djdata['date'].astype('datetime64[ns]') #convert to date
djdata.head()

,date,adj_open,adj_close,symbol
0,2019-11-15,120.98,120.76,AXP
1,2019-11-14,120.00,120.93,AXP
2,2019-11-13,120.27,120.26,AXP
3,2019-11-12,120.87,120.89,AXP
4,2019-11-11,120.76,120.90,AXP


Print Row/Col counts

In [4]:
print(djdata.shape)

(207407, 4)


Connect to database or create if it doesn't exist

In [5]:
connection_string = "postgres:postgres@localhost:5432/tbm"
engine = create_engine(f'postgresql://{connection_string}')
if not database_exists(engine.url):  #pip install sqlalchemy-utils
    create_database(engine.url)

Drop table if it exists

In [6]:
engine.execute('DROP TABLE if exists "stock_data"')

Check to be sure the table is dropped

In [7]:
engine.table_names()

[]

Create fresh table

In [8]:
from sqlalchemy import Table, Column, Integer, String, MetaData, Float, Date, ForeignKey

metadata = MetaData()
sdata = Table('stock_data', metadata,
    Column('id', Integer, primary_key=True),
    Column('date', Date),
    Column('adj_open', Float),
    Column('adj_close', Float),
    Column('symbol', String),
    Column('end_val', Float),
)

metadata.create_all(engine)


Check if table is created

In [9]:
engine.table_names()

['stock_data']

Set the index to the id field and reset the index count after concat

In [10]:
djdata.index.name = 'id'
djdata.reset_index()
djdata.head()
djdata.columns

Index(['date', 'adj_open', 'adj_close', 'symbol'], dtype='object')

In [12]:
djdata2 = pd.DataFrame()
djdata2 = djdata

yrs = []
syms = []
for yr in djdata2['date'].dt.year:
    if yr not in yrs and yr >= 2000:
        yrs.append(yr) 
#print(yrs)
for sym in djdata2['symbol']:
    if sym not in syms:
        syms.append(sym)

li2 = []
for sym in syms:
    ticker_all = djdata2[djdata['symbol'] == sym]
    for yr in yrs:
        ticker_yr = ticker_all[ticker_all['date'].dt.year == yr] 
        min_dt = min(ticker_yr['date'])
        max_dt = max(ticker_yr['date'])
        ticker_yr['date'].dtypes
        ticker_min = ticker_yr[ticker_yr['date'] == min_dt.date()]
        ticker_max = ticker_yr[ticker_yr['date'] == max_dt.date()]
        ao_min = list(ticker_min['adj_open'])[0]
        ac_min = list(ticker_min['adj_close'])[0]
        ao_max = list(ticker_max['adj_open'])[0]
        ac_max = list(ticker_max['adj_close'])[0]
        #((Close: Last Trading Day of the Year) - (Open: First Trading Day of the Year))/(Open: First Trading Day of the Year)
        ticker_min['end_val'] = ((ac_max-ao_min)/ao_min)
        ticker_max['end_val'] = ((ac_max-ao_min)/ao_min)
        df_min = pd.DataFrame(ticker_min, columns =['date', 'adj_open', 'adj_close', 'symbol','end_val'])
        df_max = pd.DataFrame(ticker_max, columns =['date', 'adj_open', 'adj_close', 'symbol','end_val'])
        li2.append(df_min)
        li2.append(df_max)

df_all = pd.concat(li2, axis=0, ignore_index=True)

df_all.index.name = 'id'
df_all.reset_index()
        
df_all.head()
        
        

C:\Users\robch\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
C:\Users\robch\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
C:\Users\robch\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,date,adj_open,adj_close,symbol,end_val
id,,,,,
0,2019-01-02,92.548131,94.292463,AXP,0.304835
1,2019-11-15,120.980000,120.760000,AXP,0.304835
2,2018-01-02,96.885143,96.117678,AXP,-0.030422
3,2018-12-31,93.750439,93.937684,AXP,-0.030422
4,2017-01-03,71.606890,72.046724,AXP,0.347316


Insert data into Postgres

In [13]:
df_all.to_sql(name='stock_data', con=engine, if_exists='append', index=True)

Show sample data

In [14]:
engine.execute('SELECT * FROM stock_data LIMIT 5').fetchall()

[(0, datetime.date(2019, 1, 2), 92.54813132, 94.29246305, 'AXP', 0.304834557733564),
 (1, datetime.date(2019, 11, 15), 120.98, 120.76, 'AXP', 0.304834557733564),
 (2, datetime.date(2018, 1, 2), 96.88514321, 96.11767843, 'AXP', -0.0304222030576074),
 (3, datetime.date(2018, 12, 31), 93.75043906, 93.93768371, 'AXP', -0.0304222030576074),
 (4, datetime.date(2017, 1, 3), 71.60689011, 72.04672413, 'AXP', 0.347316212333691)]

Check row counts

In [15]:
engine.execute('SELECT count(*) FROM stock_data').fetchall()

[(680,)]